SECTION 1: INITIAL CODE

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import MinMaxScaler

##########################

function = 6
# Read the files
X_init = np.load("initial_inputs.npy")
y_init = np.load("initial_outputs.npy")
queries_file = "queries.txt"
observations_file = "observations.txt"

# Read queries data
import ast
queries_data = []
with open(queries_file, 'r') as f:
    for line in f:
        line = line.replace('array(', 'np.array(')
        queries_data.append(eval(line.strip()))

# Read observations data
observations_data = []
with open(observations_file, 'r') as f:
    for line in f:
        observations_data.append(eval(line.strip()))

# Extract the specified sub-arrays from queries
X = np.array([q[function - 1] for q in queries_data], dtype='float64')
y = np.array([o[function - 1] for o in observations_data])

# Find and remove duplicates
unique_indices = []
seen = set()
for i, x in enumerate(X):
    x_tuple = tuple(x)  # Convert to tuple for hashability
    if x_tuple not in seen:
        seen.add(x_tuple)
        unique_indices.append(i)

# Keep only unique queries and observations
X_unique = np.concatenate((X_init, X[unique_indices]))
y_unique = np.concatenate((y_init, y[unique_indices]))
queries_unique = [queries_data[i] for i in unique_indices]
observations_unique = [observations_data[i] for i in unique_indices]

# Save cleaned data to new files
with open("queries_unique.txt", "w") as f:
    for query in queries_unique:
        f.write(str(query) + "\n")

with open("observations_unique.txt", "w") as f:
    for obs in observations_unique:
        f.write(str(obs) + "\n")

# Save cleaned numpy arrays
np.save("initial_inputs_unique.npy", X_unique)
np.save("initial_outputs_unique.npy", y_unique)

###########################

# Create DataFrame
df = pd.DataFrame({
    'param1': X_unique[:, 0],
    'param2': X_unique[:, 1],
    'param3': X_unique[:, 2],
    'param4': X_unique[:, 3],
    'param5': X_unique[:, 4],
    'output': y_unique
})

########################

import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# Black-box function for experimental evaluation
def black_box_function(x):
    # Replace with your experimental evaluation
    # Input: x = [param1, param2, param3, param4, param5] in [0, 1]^5
    # Example: Bake cake with these proportions, get negative scores, return sum
    #raise NotImplementedError("Implement experimental evaluation: bake cake and sum negative scores")
    # For testing, uncomment the synthetic function below:
    center = np.array([0.5, 0.4, 0.3, 0.6, 0.5])
    return np.sum((x - center) ** 2)

# Expected Improvement acquisition function (for minimization)
def expected_improvement(X, X_sample, y_sample, gp, xi=0.01):
    mu, sigma = gp.predict(X.reshape(-1, 5), return_std=True)
    mu_sample_opt = np.min(y_sample)  # Minimize sum of negative scores
    with np.errstate(divide='warn'):
        imp = mu_sample_opt - mu - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return -ei  # Minimize negative EI

# Bayesian Optimization using existing data frame
def bayesian_optimization(df, n_iter=10):
    # Extract features and output from data frame
    X = df[['param1', 'param2', 'param3', 'param4', 'param5']].to_numpy()
    y = df['output'].to_numpy()
    
    # Initialize Gaussian Process
    kernel = Matern(nu=2.5)
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
    gp.fit(X, y)
    
    bounds = np.array([[0, 1]] * 5)  # Inputs in [0, 1]^5
    best_x = X[np.argmin(y)]
    best_y = np.min(y)
    
    print(f"Initial best point: {best_x}, Sum of negative scores: {best_y}")
    
    for i in range(n_iter):
        # Optimize acquisition function
        x0 = np.random.uniform(0, 1, 5)  # Random starting point
        res = minimize(
            lambda x: expected_improvement(x, X, y, gp),
            x0,
            bounds=bounds,
            method='L-BFGS-B'
        )
        x_next = res.x
        
        # Evaluate black-box function (experimental)
        y_next = black_box_function(x_next)
        
        # Update data frame
        new_row = pd.DataFrame({
            'param1': [x_next[0]],
            'param2': [x_next[1]],
            'param3': [x_next[2]],
            'param4': [x_next[3]],
            'param5': [x_next[4]],
            'output': [y_next]
        })
        df = pd.concat([df, new_row], ignore_index=True)
        
        # Update X and y
        X = df[['param1', 'param2', 'param3', 'param4', 'param5']].to_numpy()
        y = df['output'].to_numpy()
        
        # Refit GP
        gp.fit(X, y)
        
        # Update best
        if y_next < best_y:
            best_x = x_next
            best_y = y_next
            print(f"Iteration {i+1}: New best point: {x_next}, Sum: {y_next}")
        else:
            print(f"Iteration {i+1}: Evaluated point: {x_next}, Sum: {y_next}")
    
    return best_x, best_y, df


best_x, best_y, updated_df = bayesian_optimization(df, n_iter=50)
print(f"\nOptimal inputs: {best_x}")
print(f"Minimum sum of negative scores: {best_y}")


Initial best point: [0.12572016 0.86272469 0.02854433 0.24660527 0.75120624], Sum of negative scores: -2.5711696316081234
Iteration 1: Evaluated point: [0.03513832 1.         0.05858456 0.14622461 1.        ], Sum: 1.090289904020251
Iteration 2: Evaluated point: [1.         1.         0.58774217 1.         0.63274729], Sum: 0.8704173995185187
Iteration 3: Evaluated point: [0.67004758 0.25510479 0.52522885 0.10438031 0.79029202], Sum: 0.4305471638129578
Iteration 4: Evaluated point: [0.22588243 0.77684957 0.03562196 0.26939103 0.61963143], Sum: 0.4106657605424981


C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 5: Evaluated point: [0.24230104 0.5746347  0.8121854  0.75729344 0.79858772], Sum: 0.4731357740376446
Iteration 6: Evaluated point: [0.83690277 0.78568768 0.37800457 0.49963128 0.68297216], Sum: 0.3118958657474306
Iteration 7: Evaluated point: [0.45550186 0.1698971  0.16587768 0.4535973  0.31368222], Sum: 0.12906429175913126
Iteration 8: Evaluated point: [0.67499165 0.7162088  0.75900934 0.26456011 0.3029492 ], Sum: 0.49264859406748374
Iteration 9: Evaluated point: [0.55509224 0.59745917 0.92124195 0.81530382 0.00710274], Sum: 0.7172702773319657
Iteration 10: Evaluated point: [0.40763016 0.74413986 0.54953152 0.19840608 0.76064954], Sum: 0.4184462656772053


C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 11: Evaluated point: [0.84721441 0.35815673 0.13141626 0.54519598 0.52674085], Sum: 0.15444773376639856
Iteration 12: Evaluated point: [6.07872733e-01 8.11198402e-01 4.44844692e-04 3.84065574e-01
 7.35058385e-01], Sum: 0.3723340634826739
Iteration 13: Evaluated point: [0.81964295 0.20657572 0.17691664 0.18597584 0.15426797], Sum: 0.4456807233029569
Iteration 14: Evaluated point: [0.62851908 0.2303332  0.89215076 0.03699681 0.67008882], Sum: 0.7418493041316577
Iteration 15: Evaluated point: [0.18055285 0.07265008 0.9530368  0.20760702 0.70273468], Sum: 0.8307351066499385


...

C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarni

Iteration 40: Evaluated point: [0.32206723 0.64301004 0.47835254 0.87143669 0.5157728 ], Sum: 0.19645023518252405
Iteration 41: Evaluated point: [0.86388789 0.03645258 0.48435346 0.03305664 0.67345084], Sum: 0.6500772865793896
Iteration 42: Evaluated point: [0.63392425 0.0803708  0.7487538  0.09134544 0.61383702], Sum: 0.593166836898591
Iteration 43: Evaluated point: [0.54448259 0.78617087 0.54811638 0.4000939  0.69183538], Sum: 0.28943163680283757
Iteration 44: Evaluated point: [0.75708666 0.70587622 0.51212123 0.62167082 0.97051267], Sum: 0.4265010290861614
Iteration 45: Evaluated point: [0.05624738 0.05291433 0.69570635 0.06241625 0.6087087 ], Sum: 0.774782237935915


C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarni

Iteration 46: Evaluated point: [0.95385883 0.83017195 0.92397467 0.93525711 0.79176728], Sum: 0.9779056114665591
Iteration 47: Evaluated point: [0.54529939 0.73625962 0.08250945 0.9683222  0.76741152], Sum: 0.3695948692280824
Iteration 48: Evaluated point: [0.75452609 0.04684134 0.90317715 0.94183359 0.94027687], Sum: 0.864021171901565
Iteration 49: Evaluated point: [0.11844805 0.151817   0.84347986 0.33159227 0.42279723], Sum: 0.5805500316172718
Iteration 50: Evaluated point: [0.54944224 0.85269627 0.76232138 0.07937229 0.17466659], Sum: 0.7980145428345927

Optimal inputs: [0.12572016 0.86272469 0.02854433 0.24660527 0.75120624]
Minimum sum of negative scores: -2.5711696316081234


C:\Users\sabri\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


SECTION 2: CODE MODIFICATION

SECTION 3: FINAL RESULT

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import MinMaxScaler
###########################

function = 6
# Read the files
X_init = np.load("initial_inputs.npy")
y_init = np.load("initial_outputs.npy")
queries_file = "queries.txt"
observations_file = "observations.txt"

# Read queries data
import ast
queries_data = []
with open(queries_file, 'r') as f:
    for line in f:
        line = line.replace('array(', 'np.array(')
        queries_data.append(eval(line.strip()))

# Read observations data
observations_data = []
with open(observations_file, 'r') as f:
    for line in f:
        observations_data.append(eval(line.strip()))

# Extract the specified sub-arrays from queries
X = np.array([q[function - 1] for q in queries_data], dtype='float64')
y = np.array([o[function - 1] for o in observations_data])

# Find and remove duplicates
unique_indices = []
seen = set()
for i, x in enumerate(X):
    x_tuple = tuple(x)  # Convert to tuple for hashability
    if x_tuple not in seen:
        seen.add(x_tuple)
        unique_indices.append(i)

# Keep only unique queries and observations
X_unique = np.concatenate((X_init, X[unique_indices]))
y_unique = np.concatenate((y_init, y[unique_indices]))
queries_unique = [queries_data[i] for i in unique_indices]
observations_unique = [observations_data[i] for i in unique_indices]

# Save cleaned data to new files
with open("queries_unique.txt", "w") as f:
    for query in queries_unique:
        f.write(str(query) + "\n")

with open("observations_unique.txt", "w") as f:
    for obs in observations_unique:
        f.write(str(obs) + "\n")

# Save cleaned numpy arrays
np.save("initial_inputs_unique.npy", X_unique)
np.save("initial_outputs_unique.npy", y_unique)
#################################

# Create DataFrame
df = pd.DataFrame({
    'param1': X_unique[:, 0],
    'param2': X_unique[:, 1],
    'param3': X_unique[:, 2],
    'param4': X_unique[:, 3],
    'param5': X_unique[:, 4],
    'output': y_unique
})
############################

from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize


# Expected Improvement acquisition function (for maximization of output)
def expected_improvement(X, X_sample, y_sample, gp, xi=0.01):
    mu, sigma = gp.predict(X.reshape(-1, 5), return_std=True)
    mu_sample_opt = np.max(y_sample)  # Maximize output (closest to zero since negative)
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return -ei  # We minimize negative EI to maximize EI

# Bayesian Optimization using existing data frame
def bayesian_optimization(df, n_iter=10):
    # Extract features and output from data frame
    X = df[['param1', 'param2', 'param3', 'param4', 'param5']].to_numpy()
    y = df['output'].to_numpy()
    
    # Initialize Gaussian Process
    kernel = Matern(nu=2.5)
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
    gp.fit(X, y)
    
    bounds = np.array([[0, 1]] * 5)  # Inputs in [0, 1]^5
    best_idx = np.argmax(y)  # Best is max y (closest to zero)
    best_x = X[best_idx]
    best_y = y[best_idx]
    
    print(f"Initial best point: {best_x}, Output (closest to zero): {best_y}")
    
    for i in range(n_iter):
        # Optimize acquisition function (maximize EI)
        x0 = np.random.uniform(0, 1, 5)  # Random starting point
        res = minimize(
            lambda x: expected_improvement(x, X, y, gp),
            x0,
            bounds=bounds,
            method='L-BFGS-B'
        )
        x_next = res.x
        
        # Predict the output for the next point
        y_next = gp.predict(x_next.reshape(1, -1))[0]
        
        # Update data frame
        new_row = pd.DataFrame({
            'param1': [x_next[0]],
            'param2': [x_next[1]],
            'param3': [x_next[2]],
            'param4': [x_next[3]],
            'param5': [x_next[4]],
            'output': [y_next]
        })
        df = pd.concat([df, new_row], ignore_index=True)
        
        # Update X and y
        X = df[['param1', 'param2', 'param3', 'param4', 'param5']].to_numpy()
        y = df['output'].to_numpy()
        
        # Refit GP
        gp.fit(X, y)
        
        # Update best
        if y_next > best_y:
            best_x = x_next
            best_y = y_next
            print(f"Iteration {i+1}: New best point: {x_next}, Output (closest to zero): {y_next}")
        else:
            print(f"Iteration {i+1}: Evaluated point: {x_next}, Output: {y_next}")
    
    # Find next best point to explore (optimize acquisition function once more)
    x0 = np.random.uniform(0, 1, 5)
    res = minimize(
        lambda x: expected_improvement(x, X, y, gp),
        x0,
        bounds=bounds,
        method='L-BFGS-B'
    )
    next_point = res.x
    next_point_pred = gp.predict(next_point.reshape(1, -1))[0]
    
    print(f"\nFinal best point in dataset: {best_x}, Output: {best_y}")
    print(f"Next best point to explore: {next_point}, Expected output: {next_point_pred}")

bayesian_optimization(df, n_iter=20) # changed the number of iterations from 10 to 20 to try to find a better value. looks like working


Initial best point: [0.475859 0.321227 0.408649 0.763953 0.09598 ], Output (closest to zero): -0.29648469308830117
Iteration 1: Evaluated point: [0.2154997  0.1468889  0.04423747 0.1627728  0.5255426 ], Output: -1.8536764091416074
Iteration 2: Evaluated point: [0.65006417 0.07880898 0.00723422 0.43022919 0.81821823], Output: -2.101450011285941
Iteration 3: New best point: [0.38074692 0.36247126 0.51364881 0.76156949 0.08385686], Output (closest to zero): -0.28929573997502245
Iteration 4: Evaluated point: [0.14839893 0.81534725 0.99289857 0.69517801 0.83088771], Output: -1.8027547521341878
Iteration 5: Evaluated point: [0.44247364 0.97370001 0.69424653 0.00302634 0.86553429], Output: -2.053930007986769
Iteration 6: Evaluated point: [0.02012605 0.44892458 0.70806426 0.31183144 0.27377265], Output: -1.1743262471309681
Iteration 7: Evaluated point: [0.65112144 0.77028198 0.5905312  0.97283533 0.33340246], Output: -1.1865184604985322
Iteration 8: Evaluated point: [0.22842966 0.95742265 0.76